# Evaluate the AUC-SMA feature

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statistics import mean, median
from random_ideas import hold_days_after_buy
import sqlite3
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline


%load_ext autoreload
%autoreload 2

db = sqlite3.connect('/Users/jluan/code/finance/fmp_data.db')

def query(sql):
  df = pd.read_sql_query(sql, db)
  return df

# A brief look at how do the AUC-SMA featurey perform

In [29]:
from feature_gen.auc_sma import AUC_SMA

stocks = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'META', 'TSM', 'CCL', 'MSCI', 'CLS', 'NVDA', 'ABNB', 'BRK-A', 'ZETA', 'AGX', 'TWLO', 'TMUS', 'SKYW', 'GM', 'CRS', 'PFG', 'SNAP', 'LYFT' ]

sorted([(symbol, 
round((100 * AUC_SMA(symbol=symbol, current_date='2024-11-01', look_back_days=300, slow_days=90, fast_days=30).calculate()),2)) for symbol in 
stocks], key=lambda x: x[1], reverse=True)
    

[('ZETA', 14.67),
 ('CRS', 11.55),
 ('NVDA', 10.8),
 ('CLS', 10.0),
 ('AGX', 9.45),
 ('SKYW', 7.29),
 ('META', 6.24),
 ('TSM', 6.19),
 ('TMUS', 4.11),
 ('AMZN', 3.46),
 ('GOOG', 2.82),
 ('BRK-A', 2.78),
 ('GM', 2.68),
 ('MSFT', 2.4),
 ('AAPL', 2.14),
 ('MSCI', 1.48),
 ('PFG', 1.28),
 ('LYFT', 1.14),
 ('CCL', 1.03),
 ('TWLO', -0.04),
 ('ABNB', -0.33),
 ('SNAP', -0.64)]

# Correlation with future price gain

In [5]:
p = query("""
select S.symbol, end_price / start_price - 1 as gain
from stock22to24 S
join (
    select symbol, adjusted_close as start_price
    from daily_price
    where date = '2023-01-05'
) AS A
join (
    select symbol, adjusted_close as end_price
    from daily_price
    where date = '2024-01-05'
) AS B
ON S.symbol = A.symbol AND S.symbol = B.symbol
""")

In [9]:
p[p.symbol == 'ABNB']

,symbol,gain
14,ABNB,0.550336
